In [1]:
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

torch.manual_seed(777) # reproducibility

In [2]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [4]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True)

In [5]:
# CNN Model
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self._build_net()
  
  def _build_net(self):
    # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
    self.keep_prob = 0.7
    # L1 ImgIn shape=(?, 28, 28, 1)
    #    Conv     -> (?, 28, 28, 32)
    #    Pool     -> (?, 14, 14, 32)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
        torch.nn.Dropout(p = 1 - self.keep_prob)
    )
    # L2 ImgIn shape=(?, 14, 14, 32)
    #    Conv      ->(?, 14, 14, 64)
    #    Pool      ->(?, 7, 7, 64)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
        torch.nn.Dropout(p = 1 - self.keep_prob)
    )
    # L3 ImgIn shape=(?, 7, 7, 64)
    #    Conv      ->(?, 7, 7, 128)
    #    Pool      ->(?, 4, 4, 128)
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1),
        torch.nn.Dropout(p = 1 - self.keep_prob)
    )
    # L4 FC 4x4x128 inputs -> 625 outputs
    self.keep_prob = 0.5
    self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias = True)
    torch.nn.init.xavier_uniform(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p = 1 - self.keep_prob)
    )
    # L5 Final FC 625 inputs -> 10 outputs
    self.fc2 = torch.nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform(self.fc2.weight)

    # define cost/loss & optimizer
    self.criterion = torch.nn.CrossEntropyLoss() # Softmax is internally computed.
    self.optimizer = torch.optim.Adam(self.parameters(), lr = learning_rate)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1) # Flatten them for FC
    out = self.fc1(out)
    out = self.fc2(out)
    return out
  
  def predict(self, x):
    self.eval()
    return self.forward(x)
  
  def get_accuracy(self, x, y):
    prediction = self.predict(x)
    correct_prediction = (torch.max(prediction.data, 1)[1] == y.data)
    self.accuracy = correct_prediction.float().mean()
    return self.accuracy
  
  def train_model(self, x, y):
    self.train()
    self.optimizer.zero_grad()
    hypothesis = self.forward(x)
    self.cost = self.criterion(hypothesis, y)
    self.cost.backward()
    self.optimizer.step()
    return self.cost

In [6]:
# instantiate CNN model
model = CNN()

<ipython-input-5-4283afccda2f>:40: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc1.weight)
<ipython-input-5-4283afccda2f>:48: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc2.weight)


In [7]:
# train my model
print('Learning started. It takes sometime.')

for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(mnist_train) // batch_size

  for i, (batch_xs, batch_ys) in enumerate(data_loader):
    X = Variable(batch_xs) # image is already size of (28x28), no reshape
    Y = Variable(batch_ys) # label is not one-hot encoded

    cost = model.train_model(X, Y)

    avg_cost += cost.data / total_batch
  
  print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost))

print('Learnig Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.265893638
[Epoch:    2] cost = 0.0825466737
[Epoch:    3] cost = 0.0629848167
[Epoch:    4] cost = 0.0544799566
[Epoch:    5] cost = 0.0483107716
[Epoch:    6] cost = 0.0462596491
[Epoch:    7] cost = 0.040448457
[Epoch:    8] cost = 0.0387122855
[Epoch:    9] cost = 0.0378944427
[Epoch:   10] cost = 0.0362074189
[Epoch:   11] cost = 0.0360632017
[Epoch:   12] cost = 0.0348793454
[Epoch:   13] cost = 0.0334904864
[Epoch:   14] cost = 0.0320705622
[Epoch:   15] cost = 0.0331564285
Learnig Finished!


In [8]:
# Test model and check accuracy
X_test = Variable(mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float())
Y_test = Variable(mnist_test.test_labels)

/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [9]:
print('Accuracy:', model.get_accuracy(X_test, Y_test))

Accuracy: tensor(0.8141)
